In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_dir = '/kaggle/input/cutmix-17march/cutmix_new/train'
test_dir = '/kaggle/input/cutmix-17march/cutmix_new/test'


img_height = 224
img_width = 224
batch_size = 32


preprocess_input = tf.keras.applications.resnet.preprocess_input


train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

for layer in resnet_base.layers:
    layer.trainable = False

x = layers.Flatten()(resnet_base.output)
x = layers.Dense(256, activation='relu')(x)
output = layers.Dense(1, activation='sigmoid')(x)

model = models.Model(inputs=resnet_base.input, outputs=output)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_generator,
                    steps_per_epoch=None,  # Set to None to run until the generator stops
                    epochs=10,
                    validation_data=test_generator,
                    validation_steps=None)  # Set to None to run until the generator stops

loss, accuracy = model.evaluate(test_generator)
print("Test Accuracy:", accuracy)


2024-03-17 10:52:39.731316: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-17 10:52:39.731502: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-17 10:52:39.926122: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 1294 images belonging to 2 classes.
Found 856 images belonging to 2 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


41/41 ━━━━━━━━━━━━━━━━━━━━ 298s 7s/step - accuracy: 0.5656 - loss: 16.6731 - val_accuracy: 0.8470 - val_loss: 0.6771
Epoch 2/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 268s 6s/step - accuracy: 0.8799 - loss: 0.4189 - val_accuracy: 0.8411 - val_loss: 0.3425
Epoch 3/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 318s 6s/step - accuracy: 0.9390 - loss: 0.1721 - val_accuracy: 0.8692 - val_loss: 0.2992
Epoch 4/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 330s 6s/step - accuracy: 0.9743 - loss: 0.0756 - val_accuracy: 0.8902 - val_loss: 0.3136
Epoch 5/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 311s 6s/step - accuracy: 0.9758 - loss: 0.0858 - val_accuracy: 0.8867 - val_loss: 0.3239
Epoch 6/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 291s 7s/step - accuracy: 0.9856 - loss: 0.0556 - val_accuracy: 0.8680 - val_loss: 0.4290
Epoch 7/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 266s 6s/step - accuracy: 0.9851 - loss: 0.0550 - val_accuracy: 0.8984 - val_loss: 0.3208
Epoch 8/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 337s 7s/step - accuracy: 0.9884 - loss: 0.0298 - val_accuracy: 0.8995 - val_loss: 0

In [6]:
image_path = '/kaggle/input/image-cutmix-predict/testing_cutmix.png'

img = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_height, img_width))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

prediction = model.predict(img_array)

if prediction < 0.5:
    print("Original Image")
else:
    print("AI Generated Image")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
AI Generated Image
